# Parallel Evolution algorithms
A concise demonstration of the evolutionary algorithms used in the circles example:  
- MAP Elite
- The Island Model


## Map Elite
- [original paper](https://arxiv.org/pdf/1504.04909)
- [PGA-MAP-Elites repo](https://github.com/ollebompa/PGA-MAP-Elites)

The MAP-Elites (Multi-dimensional Archive of Phenotypic Elites) algorithm is an evolutionary search algorithm designed to explore a diverse set of high-performing solutions, rather than optimizing for a single best solution. It does this by organizing the search space into a map of niches, where each niche contains the best (elite) solution found for a specific region of a behavior or feature space.

**Core Principles of MAP-Elites**
1. Feature Descriptor Space (Behavior Space)  
Instead of only considering the performance (fitness) of solutions, MAP-Elites also characterizes them by a set of features or descriptors.
    - These features define the axes of a low-dimensional feature space (e.g., walking speed vs. stability in robot locomotion).
    - The space is discretized into a grid or map, with each cell representing a niche for a specific range of feature values.

2. Archive of Elites  
The algorithm maintains an archive (map) where each cell holds the elite individual (i.e., best fitness) found for that region of feature space.
    - Only the best-performing individual for each cell is kept.
    - This allows MAP-Elites to store and improve many diverse, high-quality solutions simultaneously.

3. Illumination over Optimization  
Rather than searching for one global optimum, MAP-Elites aims to illuminate the search space by revealing what high-performing solutions exist across different regions of the behavior space.
    - This makes it valuable for problems where diverse trade-offs or multiple useful strategies are important.

4. Variation and Insertion  
The algorithm iteratively performs:
    - Selection: Randomly selects individuals from the map (not necessarily the best overall).
    - Variation: Applies evolutionary operators (e.g., mutation, crossover) to create new offspring.
    - Evaluation: Measures both fitness and feature descriptors.
    - Insertion: Places the new individual in the appropriate cell of the map if:
        - That cell is empty, or
        - The individual has better fitness than the current elite in that cell.

5. Exploration through Diversity
Because MAP-Elites explores many areas of the feature space, it promotes diversity and robustness. This is in contrast to traditional evolutionary algorithms that may converge prematurely to a single optimum.

**Summary of Algorithm Steps**
1. Initialize the map (empty or seeded with random solutions).
1. Evaluate initial individuals and insert them into the map.
1. Repeat:
    - Select an elite from the map at random.
    - Mutate or recombine to produce a new solution.
    - Evaluate its fitness and features.
    - Insert into the map if it improves the fitness for its feature cell.

**Applications**
MAP-Elites is useful in domains where:
- Multiple behaviors are desirable (e.g., robot control, design optimization).
- Understanding the trade-off space is more important than finding a single best solution.
- There is deceptive or multi-modal fitness landscape.



```python
# MAP-Elites pseudocode

# --- Parameters ---
feature_dims = (f1_bins, f2_bins, ...)   # Discretization of feature space
num_initial_samples = 100
num_iterations = 10000

# --- Archive Initialization ---
archive = {}  # key: tuple of feature cell indices, value: individual (elite)

# --- Helper Functions (to be defined for your problem) ---
def generate_random_solution():
    # Create a random individual (e.g., random parameters)
    pass

def evaluate(individual):
    # Returns: (fitness_score, feature_descriptor)
    pass

def get_cell_indices(feature_descriptor):
    # Map continuous descriptor to discrete cell indices in archive
    pass

def mutate(individual):
    # Apply mutation or crossover
    pass

# --- Initial Population ---
for _ in range(num_initial_samples):
    ind = generate_random_solution()
    fitness, descriptor = evaluate(ind)
    cell = get_cell_indices(descriptor)

    if cell not in archive or fitness > archive[cell]["fitness"]:
        archive[cell] = {"individual": ind, "fitness": fitness}

# --- Main Loop ---
for _ in range(num_iterations):
    # 1. Select a parent randomly from archive
    parent_cell = random.choice(list(archive.keys()))
    parent = archive[parent_cell]["individual"]

    # 2. Variation
    offspring = mutate(parent)

    # 3. Evaluation
    fitness, descriptor = evaluate(offspring)
    cell = get_cell_indices(descriptor)

    # 4. Insertion (replace only if better)
    if cell not in archive or fitness > archive[cell]["fitness"]:
        archive[cell] = {"individual": offspring, "fitness": fitness}

# --- Output: the filled archive of elites
return archive

```

**Notes:**
- `archive` is a map of elites, keyed by discretized feature bins.
- `evaluate()` must return both fitness and feature descriptors for the individual.
- `get_cell_indices()` maps the descriptors to the appropriate discrete cell.
- The algorithm is agnostic to the solution encoding — works for controllers, neural nets, parameter vectors, etc.
- Optionally, you can visualize the archive as a heatmap if it's 2D.



## The Island Model  

Description from ChatGPT:

The Island Model is a popular parallel evolutionary algorithm framework used to improve the performance of evolutionary search methods. The core idea is to split the overall population into several subpopulations, or islands, which evolve more or less independently, with occasional communication (migration) between them.

**Core Principles of the Island-Based Population Model:**
1. Multiple Subpopulations (Islands) - 
    - Instead of evolving one large population, the algorithm maintains multiple smaller populations (islands).
    - Each island evolves independently using a local evolutionary algorithm (e.g., genetic algorithm, evolution strategy).

2. Isolation and Diversity
    - Islands are (initially) isolated from each other, which allows them to explore different parts of the search space.
    - This promotes genetic diversity and helps prevent premature convergence to suboptimal solutions.

3. Migration
    - At predefined intervals (called migration intervals), some individuals are exchanged between islands.
    - Typically, the best individuals from one island are sent to others to share good genetic material.
    - Migration can follow different topologies (e.g., ring, fully connected, random) that determine which islands exchange individuals.

4. Asynchronous or Synchronous Evolution
    - Synchronous: All islands pause for migration at the same time.
    - Asynchronous: Each island evolves and migrates on its own schedule, promoting better load balancing in parallel environments.

5. Local Adaptation with Global Sharing
    - Each island may specialize in a different region of the fitness landscape.
    - Migration allows islands to share discoveries, combining local adaptation with global information exchange.

6. Scalability and Parallelism
    - The model is naturally parallelizable since islands can run on separate processors or machines.
    - This makes it suitable for distributed or high-performance computing contexts.

**Benefits of the Island Model:**
- Improved exploration and exploitation balance
- Reduced risk of getting stuck in local optima
- Enhanced scalability and efficiency on parallel architectures
- Flexibility in algorithm design and parameter tuning per island

Use Cases:
- Large and complex optimization problems
- Multi-objective optimization
- Dynamic or noisy environments where maintaining diversity is critical

Example : lets optimize this function
```python
f(x) = x * sin(10 * π * x) + 1.0  for x ∈ [0, 1]
```

**Key Parameters**
- `num_islands`: Number of subpopulations.
- `migration_interval`: Number of generations between migrations.
- `migration_size`: Number of individuals to exchange.
- `population_size`: Number of individuals per island.
- `generations`: Number of generations per island.


In [1]:
import numpy as np

# Objective function
def fitness_function(x):
    return x * np.sin(10 * np.pi * x) + 1.0

# Individual representation
def create_individual():
    return np.random.rand()

def mutate(x, mutation_rate=0.1):
    return np.clip(x + mutation_rate * np.random.randn(), 0.0, 1.0)

def crossover(a, b):
    alpha = np.random.rand()
    return alpha * a + (1 - alpha) * b

def select_parents(population, fitnesses, num_parents):
    idx = np.argsort(fitnesses)[-num_parents:]
    return [population[i] for i in idx]

# Run GA on one island
def evolve_island(population, generations, mutation_rate):
    for _ in range(generations):
        fitnesses = [fitness_function(ind) for ind in population]
        parents = select_parents(population, fitnesses, len(population)//2)

        offspring = []
        while len(offspring) < len(population):
            a, b = np.random.choice(parents, 2)
            child = crossover(a, b)
            child = mutate(child, mutation_rate)
            offspring.append(child)

        population = offspring
    return population

# Migrate best individuals between islands
def migrate(islands, migration_size):
    num_islands = len(islands)
    migrants = []

    # Select top individuals from each island
    for island in islands:
        fitnesses = [fitness_function(ind) for ind in island]
        top_indices = np.argsort(fitnesses)[-migration_size:]
        migrants.append([island[i] for i in top_indices])

    # Distribute migrants to neighboring islands
    for i in range(num_islands):
        target_island = (i + 1) % num_islands  # ring topology
        islands[target_island][:migration_size] = migrants[i]

# Main loop
def run_island_model(num_islands=4, population_size=20, generations=50,
                     migration_interval=10, migration_size=2, mutation_rate=0.05):
    islands = [[create_individual() for _ in range(population_size)] for _ in range(num_islands)]

    for gen in range(0, generations, migration_interval):
        for i in range(num_islands):
            islands[i] = evolve_island(islands[i], migration_interval, mutation_rate)
        migrate(islands, migration_size)

    # Final results
    all_individuals = [ind for island in islands for ind in island]
    best = max(all_individuals, key=fitness_function)
    print(f"Best solution found: x = {best:.4f}, f(x) = {fitness_function(best):.4f}")

# Run it
run_island_model(num_islands=4, population_size=20, generations=50, migration_interval=10, migration_size=2, mutation_rate=0.05)


Best solution found: x = 0.6535, f(x) = 1.6496


In [6]:
run_island_model(num_islands=10, population_size=20, generations=50, migration_interval=10, migration_size=2, mutation_rate=0.05)

Best solution found: x = 0.8534, f(x) = 1.8486
